# Economic Regressions

#### Imports

In [7]:
import os
import ast
import json
import shutil
import platform
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col

tqdm.pandas()
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'

TABLES = 'Output/Tables'

#### Loading

In [8]:
with open(f'{PROCESSED}/pretty_dict.json', 'r', encoding='utf-8') as f:
    pretty_dict = json.load(f)
# %% Load data
df = pd.read_csv(f'{PROCESSED}/parish_dataset.csv', encoding='utf-8')

df.loc[pd.isna(df['ind_1831']), 'agr_1831'] = np.nan

# Set np.nan if ind_1831 == agr_1831 == 0
df.loc[(df['ind_1831'] == 0) & (df['agr_1831'] == 0), 'ind_1831'] = np.nan
df.loc[(df['ind_1831'] == 0) & (df['agr_1831'] == 0), 'agr_1831'] = np.nan
df['other_1831'] = 1 - df['ind_1831'] - df['agr_1831']
df['lland'] = np.log(df['landOwned'] + 1)
df['lland_area'] = df['lland'] / df['area']
df['monast_land_share'] = (df['landOwned'] / 240) / (df['parish_val_1524'] + df['landOwned'] / 240)

# Replace "value_1840" with "val_1840" in column names
df.rename(columns=lambda x: x.replace('value_1840', 'val_1840') if 'value_1840' in x else x, inplace=True)

In [6]:
a = list(df.columns)
#a = [x for x in a if 'Pop' in x]
a


['parish',
 'parish_val_1524',
 'parish_val_1543',
 'parish_val_1581',
 'parish_val_1674',
 'parish_count_1524',
 'parish_count_1543',
 'parish_count_1581',
 'parish_count_1674',
 'parish_max_val_1524',
 'parish_max_val_1543',
 'parish_max_val_1581',
 'parish_max_val_1674',
 'parish_unique_treatment_1524',
 'parish_unique_treatment_1543',
 'parish_unique_treatment_1581',
 'parish_unique_treatment_1674',
 'parish_unique_control_1524',
 'parish_unique_control_1543',
 'parish_unique_control_1581',
 'parish_unique_control_1674',
 'parish_unique_treatment_val_1524',
 'parish_unique_treatment_val_1543',
 'parish_unique_treatment_val_1581',
 'parish_unique_treatment_val_1674',
 'parish_unique_control_val_1524',
 'parish_unique_control_val_1543',
 'parish_unique_control_val_1581',
 'parish_unique_control_val_1674',
 'parish_group_treatment_1524',
 'parish_group_treatment_1543',
 'parish_group_treatment_1581',
 'parish_group_treatment_1674',
 'parish_group_control_1524',
 'parish_group_control_

#### Ag/Ind/Other Share Regressions

#### Treatment and Control Wealth Shares

In [15]:
for year in [1524, 1543, 1581, 1674, 1840]:
    df[f'parish_master_treatment_val_share_{year}'] = df[f'parish_master_treatment_val_{year}'] / df[f'parish_val_{year}']
    df[f'parish_master_control_val_share_{year}'] = df[f'parish_master_control_val_{year}'] / df[f'parish_val_{year}']
    result_list = []
    for depvar in ['ind_1831', 'agr_1831', 'other_1831']:
        xvars = ['monast_land_share', f'parish_master_treatment_val_share_{year}', f'parish_master_control_val_share_{year}', 
                 'mean_slope', 'wheatsuit', 'area', 'lspc1525', 'distmkt']
        reg_df = df.copy()
        reg_df = reg_df.dropna(subset=['ind_1831', 'agr_1831', 'other_1831'] + xvars)

        y = reg_df[depvar]
        y.rename(pretty_dict, inplace=True)
        x = reg_df[xvars]
        x = sm.add_constant(x)
        x.rename(columns=pretty_dict, inplace=True)
        # Fit the model with heteroskedasticity robust standard errors
        model = sm.OLS(y, x).fit(cov_type='HC3')
        result_list.append(model)
    ag_ind_reg= summary_col(result_list, stars=True,
                                float_format='%0.4f',
                                model_names=['Industry Share', 'Agriculture Share', 'Other Share'],
                                info_dict={'N': lambda x: "{0:d}".format(int(x.nobs))},
                                regressor_order=[pretty_dict[x] for x in xvars]
                                ).as_latex()
    ag_ind_reg = ag_ind_reg.replace('\\end{center}\n\\end{table}\n\\bigskip\nStandard errors in parentheses. \\newline \n* p<.1, ** p<.05, ***p<.01', '\\end{center}\n* p<.1, ** p<.05, ***p<.01\n\\end{table}')
    ag_ind_reg = ag_ind_reg.replace('<', '$<$').replace('>', '$>$')
    ag_ind_reg = ag_ind_reg.replace('\\begin{center}', '\\begin{center}\n\\resizebox{\\textwidth}{!}{')
    ag_ind_reg = ag_ind_reg.replace('\\end{tabular}', '\\end{tabular}\n}')
    ag_ind_reg = ag_ind_reg.replace('\\begin{table}', '\\begin{table}[H]')

    ag_ind_reg = ag_ind_reg.replace('{llllll}', '{llll|l|l}')
    print(ag_ind_reg)
    with open(f'{TABLES}/ag_ind_reg_val_share_{year}.tex', 'w', encoding='utf-8') as f:
        f.write(ag_ind_reg)

\begin{table}[H]
\caption{}
\label{}
\begin{center}
\resizebox{\textwidth}{!}{
\begin{tabular}{llll}
\hline
                                            & Industry Share & Agriculture Share & Other Share  \\
\hline
Monastic Land Share                         & 0.0135         & -0.2024           & 0.1889       \\
                                            & (0.1206)       & (0.2065)          & (0.1711)     \\
Treatment Group Share of Parish Wealth 1524 & -0.3866        & 0.7134            & -0.3268      \\
                                            & (0.3431)       & (0.5343)          & (0.3482)     \\
Control Group Share of Parish Wealth 1524   & -0.2210        & -0.3907           & 0.6117       \\
                                            & (0.3434)       & (0.6176)          & (0.4430)     \\
Mean Slope                                  & -0.0016        & 0.0023            & -0.0007      \\
                                            & (0.0035)       & (0.0063)          & (0.0044)  

#### Treatment and Control Surname Shares

In [16]:
for year in [1524, 1543, 1581, 1674, 1840]:
    result_list = []
    for depvar in ['ind_1831', 'agr_1831', 'other_1831']:
        xvars = ['monast_land_share', f'parish_master_treatment_{year}', f'parish_master_control_{year}', 
                 'mean_slope', 'wheatsuit', 'area', 'lspc1525', 'distmkt']
        reg_df = df.copy()
        reg_df = reg_df.dropna(subset=['ind_1831', 'agr_1831', 'other_1831'] + xvars)

        y = reg_df[depvar]
        y.rename(pretty_dict, inplace=True)
        x = reg_df[xvars]
        x = sm.add_constant(x)
        x.rename(columns=pretty_dict, inplace=True)
        # Fit the model with heteroskedasticity robust standard errors
        model = sm.OLS(y, x).fit(cov_type='HC3')
        result_list.append(model)
    ag_ind_reg= summary_col(result_list, stars=True,
                                float_format='%0.4f',
                                model_names=['Industry Share', 'Agriculture Share', 'Other Share'],
                                info_dict={'N': lambda x: "{0:d}".format(int(x.nobs))},
                                regressor_order=[pretty_dict[x] for x in xvars]
                                ).as_latex()
    ag_ind_reg = ag_ind_reg.replace('\\end{center}\n\\end{table}\n\\bigskip\nStandard errors in parentheses. \\newline \n* p<.1, ** p<.05, ***p<.01', '\\end{center}\n* p<.1, ** p<.05, ***p<.01\n\\end{table}')
    ag_ind_reg = ag_ind_reg.replace('<', '$<$').replace('>', '$>$')
    ag_ind_reg = ag_ind_reg.replace('\\begin{center}', '\\begin{center}\n\\resizebox{\\textwidth}{!}{')
    ag_ind_reg = ag_ind_reg.replace('\\end{tabular}', '\\end{tabular}\n}')
    ag_ind_reg = ag_ind_reg.replace('\\begin{table}', '\\begin{table}[H]')

    ag_ind_reg = ag_ind_reg.replace('{llllll}', '{llll|l|l}')
    print(ag_ind_reg)
    with open(f'{TABLES}/ag_ind_reg_surname_share_{year}.tex', 'w', encoding='utf-8') as f:
        f.write(ag_ind_reg)

\begin{table}[H]
\caption{}
\label{}
\begin{center}
\resizebox{\textwidth}{!}{
\begin{tabular}{llll}
\hline
                                              & Industry Share & Agriculture Share & Other Share  \\
\hline
Monastic Land Share                           & 0.0209         & -0.2165           & 0.1956       \\
                                              & (0.1212)       & (0.2131)          & (0.1749)     \\
Treatment Group Share of Parish Surnames 1524 & -0.0279        & 0.0162            & 0.0117       \\
                                              & (0.1007)       & (0.1706)          & (0.1141)     \\
Control Group Share of Parish Surnames 1524   & -0.0084        & -0.1924           & 0.2008       \\
                                              & (0.1052)       & (0.1924)          & (0.1441)     \\
Mean Slope                                    & -0.0016        & 0.0024            & -0.0008      \\
                                              & (0.0036)       & (0.0063)    